# デルタレイク クイックスタート Delta Lake Quickstart
https://docs.delta.io/latest/quick-start.html#python

# 本サンプルの目的 Ovjective
データのDELETE & UPSERT例<br>
ソースデータにないレコードの物理削除を想定している<br>
Example of DELETE & UPSERT of data<br>
Assumes physical deletion

# データ概要 Data Summary
## ターゲットデータ Target data
`/workspace/csv/vehicles_10.csv`<br>
ターゲットデータは、永続化されたデルタテーブルで、Lakehouseとしての用途を想定している<br>
Target data is a persistent delta table, intended for use as a Lakehouse
| id | vin | name | manufacturer | type | model | fuel | color | description |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | WDR5FBN83YTF11274 | Audi A8 | Dodge | Wagon | Prius | Electric | tan | still |
| 2 | E66LYTEHD8EM39881 | Maserati Alpine | Mazda | Coupe | Sentra | Gasoline | white | still |
| 3 | SM5T15RCMUX914093 | Porsche Volt | Lamborghini | Coupe | Element | Hybrid | blue | still |
| 4 | APWJUTZNJ1ST99631 | Porsche Beetle | Honda | Crew Cab Pickup | Model 3 | Hybrid | orchid | delete |
| 5 | K7K9L4AE3MLA82628 | Lamborghini Challenger | Mazda | Hatchback | XC90 | Diesel | blue | update |
| 6 | AW2B2LL9VLWJ86253 | Kia Volt | Jeep | Minivan | Mustang | Hybrid | pink | delete |
| 7 | UJSYPGEKULZE52088 | Kia Golf | Ford | Minivan | LeBaron | Hybrid | silver | update |
| 8 | W02HDVA34UPF70822 | Ford Challenger | Polestar | Hatchback | Mercielago | Electric | orchid | delete |
| 9 | GUC81JZZX1NB46693 | Land Rover Grand Cherokee | Mini | Sedan | Grand Cherokee | Hybrid | grey | update |
| 10 | 46WJ12P00MHJ64331 | BMW Altima | Mercedes Benz | Hatchback | Altima | Gasoline | orchid | delete |

## ソースデータ Source data
`/workspace/csv/vehicles_13.csv`<br>
ソースデータは、ELTの際に、ターゲットデータにデータを挿入する前にデータを挿入する一時テーブルとしての用途を想定している<br>
一時テーブルなので、使い終わったら削除をしたいが、アンマネージドテーブルは削除ができないので要注意<br>
※マネージドテーブルだと削除ができる<br>
Source data is intended to be used as a temporary table to insert data before inserting data into target data during ELT<br>
Since it is a temporary table, you want to delete it when you are done using it, but be aware that unmanaged tables cannot be deleted.<br>
※Managed table allows deletion.
| id | vin | name | manufacturer | type | model | fuel | color | description |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | WDR5FBN83YTF11274 | Audi A8 | Dodge | Wagon | Prius | Electric | tan | still |
| 2 | E66LYTEHD8EM39881 | Maserati Alpine | Mazda | Coupe | Sentra | Gasoline | white | still |
| 3 | SM5T15RCMUX914093 | Porsche Volt | Lamborghini | Coupe | Element | Hybrid | blue | still |
| 5 | K7K9L4AE3MLA82628 | Lamborghini Challenger | Mazda | 1 | 1 | 1 | 1 | update |
| 7 | UJSYPGEKULZE52088 | Kia Golf | Ford | 1 | 1 | 1 | 1 | update |
| 9 | GUC81JZZX1NB46693 | Land Rover Grand Cherokee | Mini | 1 | 1 | 1 | 1 | update |
| 11 | CMMSJY81W1G260371 | Mini V90 | Maserati | Hatchback | Camry | Gasoline | mint green | insert |
| 12 | B3AMC4Z9V1PU82625 | Kia V90 | Toyota | Coupe | Golf | Diesel | ivory | insert |
| 13 | SC6J9XU0G5Z534144 | Chevrolet Model Y | Mercedes Benz | Wagon | Charger | Electric | magenta | insert |

## UPSERT後のデータ Data after UPSERT
`/workspace/tables/vehicles`<br
ターゲットデータにソースデータをDELETE & UPSERTした結果、得られる予定のデータ<br>
Data expected to result from DELETE & UPSERT of source data to target data
| id | vin | name | manufacturer | type | model | fuel | color | description |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 1 | WDR5FBN83YTF11274 | Audi A8 | Dodge | Wagon | Prius | Electric | tan | still |
| 2 | E66LYTEHD8EM39881 | Maserati Alpine | Mazda | Coupe | Sentra | Gasoline | white | still |
| 3 | SM5T15RCMUX914093 | Porsche Volt | Lamborghini | Coupe | Element | Hybrid | blue | still |
| 5 | K7K9L4AE3MLA82628 | Lamborghini Challenger | Mazda | 1 | 1 | 1 | 1 | update |
| 7 | UJSYPGEKULZE52088 | Kia Golf | Ford | 1 | 1 | 1 | 1 | update |
| 9 | GUC81JZZX1NB46693 | Land Rover Grand Cherokee | Mini | 1 | 1 | 1 | 1 | update |
| 11 | CMMSJY81W1G260371 | Mini V90 | Maserati | Hatchback | Camry | Gasoline | mint green | insert |
| 12 | B3AMC4Z9V1PU82625 | Kia V90 | Toyota | Coupe | Golf | Diesel | ivory | insert |
| 13 | SC6J9XU0G5Z534144 | Chevrolet Model Y | Mercedes Benz | Wagon | Charger | Electric | magenta | insert |

# 処理イメージ
```mermaid
flowchart TB
  TD([name : Target data\npath : /workspace/csv/vehicles_10.csv\ntype : csv])
  SD([name : Source data\npath : /workspace/csv/vehicles_15.csv\ntype : csv])

  DFTD[name : csvDf\ntype : dataformat]
  DFSD[name : csvDfTmp\ntype : dataformat]

  DTTD[(name : products\npath : /workspace/tables/vehicles\ntype : Delta table)]
  DTSD[(products_tmp\n/workspace/tables/vehicles_tmp\ntype : Delta table)]

  TD-->|spark.read|DFTD
  SD-->|spark.read|DFSD

  DFTD-->|write\noverwrite|DTTD
  DFSD-->|write\noverwrite|DTSD

  DTSD-->|delete & upsert\nkey = id\n\nソースデータにないレコードを\n物理削除している|DTTD
```
※処理フローは要件によって変わってくるので、あくまでも参考とすること<br>
The processing flow will vary depending on requirements and should be used as a reference only.

In [1]:
import pyspark
from delta import *
from delta.tables import *
import packages.modules as modules
from pyspark.sql.functions import *
from pyspark.sql.types import StringType, StructField, StructType

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
  .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
  .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

workspace = "/workspace"
file_name = "vehicles_10"
file_name_tmp = "vehicles_13"
file_ext = ".csv"
data_path = workspace + "/csv/" + file_name + file_ext
data_path_tmp = workspace + "/csv/" + file_name_tmp + file_ext
delta_table_name = "vehicles"
delta_table_name_tmp = "vehicles_tmp"
delta_table_path = workspace + "/tables/" + delta_table_name
delta_table_path_tmp = workspace + "/tables/" + delta_table_name_tmp

:: loading settings :: url = jar:file:/usr/local/lib/python3.11/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/vscode/.ivy2/cache
The jars for the packages stored in: /home/vscode/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-771aaf88-f616-4a3c-bb94-b8a325de37f5;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 140ms :: artifacts dl 9ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0 

# 処理開始
vehicles_10のcsvデータを取得する

In [2]:
csvDf = spark.read.option("delimiter", ",").option("header", "true").csv(data_path)
csvDf.printSchema()
csvDf.show(100)

root
 |-- id: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- name: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- type: string (nullable = true)
 |-- model: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- color: string (nullable = true)
 |-- description: string (nullable = true)

+---+-----------------+--------------------+-------------+---------------+--------------+--------+------+-----------+
| id|              vin|                name| manufacturer|           type|         model|    fuel| color|description|
+---+-----------------+--------------------+-------------+---------------+--------------+--------+------+-----------+
|  1|WDR5FBN83YTF11274|             Audi A8|        Dodge|          Wagon|         Prius|Electric|   tan|      still|
|  2|E66LYTEHD8EM39881|     Maserati Alpine|        Mazda|          Coupe|        Sentra|Gasoline| white|      still|
|  3|SM5T15RCMUX914093|        Porsche Volt|  Lamborghini|        

product_15のcsvデータを取得する

In [3]:
csvDfTmp = spark.read.option("delimiter", ",").option("header", "true").csv(data_path_tmp)
csvDfTmp.printSchema()
csvDfTmp.show(100)

root
 |-- id: string (nullable = true)
 |-- vin: string (nullable = true)
 |-- name: string (nullable = true)
 |-- manufacturer: string (nullable = true)
 |-- type: string (nullable = true)
 |-- model: string (nullable = true)
 |-- fuel: string (nullable = true)
 |-- color: string (nullable = true)
 |-- description: string (nullable = true)

+---+-----------------+--------------------+-------------+---------+-------+--------+----------+-----------+
| id|              vin|                name| manufacturer|     type|  model|    fuel|     color|description|
+---+-----------------+--------------------+-------------+---------+-------+--------+----------+-----------+
|  1|WDR5FBN83YTF11274|             Audi A8|        Dodge|    Wagon|  Prius|Electric|       tan|      still|
|  2|E66LYTEHD8EM39881|     Maserati Alpine|        Mazda|    Coupe| Sentra|Gasoline|     white|      still|
|  3|SM5T15RCMUX914093|        Porsche Volt|  Lamborghini|    Coupe|Element|  Hybrid|      blue|      still|
| 

# デルタテーブルにデータを書き込む
ローカル環境上で`saveAsTable`を使おうとすると、エラーが発生する。<br>
原因は、今の所、調査中。<br>
なので、`save`を使って、データを書き込んでいる。

In [4]:
csvDf.write\
  .format("delta")\
  .mode("overwrite")\
  .partitionBy("type")\
  .save(delta_table_path)

In [5]:
csvDfTmp.write\
  .format("delta")\
  .mode("overwrite")\
  .partitionBy("type")\
  .save(delta_table_path_tmp)

# デルタテーブルをDelete & Upsertする

In [6]:
deltaTable = DeltaTable.forPath(spark, delta_table_path)
deltaTableTmp = DeltaTable.forPath(spark, delta_table_path_tmp)
dfTemp = deltaTableTmp.toDF()

deltaTable.alias("vehicles")\
  .merge(
    source = dfTemp.alias("tableTmp"),
    condition = "vehicles.id = tableTmp.id"
  )\
  .whenMatchedUpdateAll()\
  .whenNotMatchedInsertAll()\
  .whenNotMatchedBySourceDelete()\
  .execute()

# デルタテーブルを読み込む

In [7]:
df = spark.read.format("delta").load(delta_table_path)
df.createOrReplaceTempView(delta_table_name)
spark.conf.set('dq.val.delta_table_name', delta_table_name)
spark.sql(
  """
    SELECT
      id,
      vin,
      name,
      manufacturer,
      type,
      model,
      fuel,
      color,
      description
    FROM ${dq.val.delta_table_name}
    ORDER BY CAST(id AS BIGINT) ASC
  """
).show(100)

+---+-----------------+--------------------+-------------+---------+-------+--------+----------+-----------+
| id|              vin|                name| manufacturer|     type|  model|    fuel|     color|description|
+---+-----------------+--------------------+-------------+---------+-------+--------+----------+-----------+
|  1|WDR5FBN83YTF11274|             Audi A8|        Dodge|    Wagon|  Prius|Electric|       tan|      still|
|  2|E66LYTEHD8EM39881|     Maserati Alpine|        Mazda|    Coupe| Sentra|Gasoline|     white|      still|
|  3|SM5T15RCMUX914093|        Porsche Volt|  Lamborghini|    Coupe|Element|  Hybrid|      blue|      still|
|  5|K7K9L4AE3MLA82628|Lamborghini Chall...|        Mazda|        1|      1|       1|         1|     update|
|  7|UJSYPGEKULZE52088|            Kia Golf|         Ford|        1|      1|       1|         1|     update|
|  9|GUC81JZZX1NB46693|Land Rover Grand ...|         Mini|        1|      1|       1|         1|     update|
| 11|CMMSJY81W1G260